# Fixed-Income Instruments and Risk Metrics: Duration & Convexity

This notebook covers **fixed-income instruments** and their associated **risk metrics**. It describes the common cash flow structures of various fixed-income instruments and the essential metrics used to evaluate them, focusing on duration and convexity.


### Key Topics:

1. **Cash Flow Structures**:
   - **Bullet Bond**: A bond that pays periodic interest and a lump sum at maturity.
   - **Fully Amortizing Loan**: A loan where the principal is paid down over time, typically through equal periodic payments.
   - **Asset-Backed Security (ABS)**: A financial security backed by a pool of assets, such as loans, leases, or receivables.

2. **Fixed Income Metrics**:
   - **Duration**: A measure of the weighted average time until a bond's cash flows are received, used to assess interest rate risk.
   - **Convexity**: A measure of the sensitivity of a bond's duration to changes in interest rates, providing a more accurate assessment of interest rate risk for larger changes in yield.

### Highlights

This notebook emphasizes the use of Object-Oriented Programming (OOP) concepts to model and analyze fixed-income instruments. Key OOP principles demonstrated include:

- **Inheritance**: Using a base class (`FixedIncomeInstrument`) to define common properties and methods, which are then inherited by specific instrument classes.

- **Abstract Base Classes (ABC)**: Defining an abstract base class to enforce a common interface for all fixed-income instruments.

- **Encapsulation**: Encapsulating the data and behavior of each instrument within its respective class, ensuring modular and maintainable code.

- **Generators**: Utilizing generators for efficient iteration over cash flows and other sequences, enhancing performance and readability.




## Fixed‐Income Instruments Module:

 
- An abstract base class (`FixedIncomeInstrument`)  
- Three concrete classes: `BulletBond`, `AmortizingLoan`, `AssetBackedSecurity`  
- A script at the bottom demonstrating pricing & YTM for different fixed-income instruments.
- Functions to calculate bond duration and convexity.
- A sample test demonstrating the calculation of duration and convexity.

## classes:

In [3]:
# ──────────────────────────────────────────────────────────────
# 1) Abstract Base Class
# ──────────────────────────────────────────────────────────────
from abc import ABC, abstractmethod
from datetime import date, timedelta

class FixedIncomeInstrument(ABC):
    """Abstract Base Class for any fixed-income instrument."""
    
    @abstractmethod
    def cash_flows(self) -> list[tuple[date, float]]:
        """
        Return a list of (payment_date, amount) tuples representing
        the instrument’s scheduled cash flows.
        """
        pass
    
    @abstractmethod
    def price(self, yield_rate: float, as_of: date = date.today()) -> float:
        """
        Discount the cash flows at yield_rate (annual) back to as_of date
        and return the present value.
        """
        pass
    
    def yield_to_maturity(self, market_price: float, guess: float = 0.05) -> float:
        """
        Compute the yield that makes price(yield) == market_price,
        via a naive Newton–Raphson.
        """
        y = guess
        for _ in range(20):
            p  = self.price(y)
            dy = 1e-6
            p2 = self.price(y + dy)
            dpdy = (p2 - p) / dy
            y  -= (p - market_price) / dpdy
        return y


In [4]:
# ──────────────────────────────────────────────────────────────
# 2) BulletBond
# ──────────────────────────────────────────────────────────────
class BulletBond(FixedIncomeInstrument):
    """
    A simple bullet bond: pays periodic coupons, pays par at maturity
    """
    def __init__(self, par: float, coupon_rate: float,
                 maturity: date, freq: int = 2):
        self.par         = par
        self.coupon_rate = coupon_rate
        self.maturity    = maturity
        self.freq        = freq

    def cash_flows(self) -> list[tuple[date, float]]:
        flows = []
        dt = 365 // self.freq
        pay_date = self.maturity

        # generate backwards coupon dates until today
        while pay_date > date.today():
            coupon = self.par * self.coupon_rate / self.freq
            flows.append((pay_date, coupon))
            pay_date -= timedelta(days=dt)

        # add principal to the earliest flow
        flows[0] = (flows[0][0], flows[0][1] + self.par)
        return sorted(flows)

    def price(self, yield_rate: float, as_of: date = date.today()) -> float:
        pv = 0.0
        for pay_date, amt in self.cash_flows():
            t = (pay_date - as_of).days / 365
            pv += amt / ((1 + yield_rate) ** t)
        return pv


In [6]:
# ──────────────────────────────────────────────────────────────
# 3) AmortizingLoan
# ──────────────────────────────────────────────────────────────
class AmortizingLoan(FixedIncomeInstrument):
    """
    A fully-amortizing loan:level payments of interest + principal, term in years, frequency per year
    """
    def __init__(self, principal: float, rate: float,
                 term_years: int, freq: int = 12):
        self.principal = principal
        self.rate      = rate
        self.term      = term_years
        self.freq      = freq

    def payment_amount(self) -> float:
        r = self.rate / self.freq
        n = self.term * self.freq
        return (self.principal * r) / (1 - (1 + r) ** -n)

    def cash_flows(self) -> list[tuple[date, float]]:
        pmt     = self.payment_amount()
        flows   = []
        balance = self.principal
        today   = date.today()

        for i in range(1, self.term * self.freq + 1):
            pay_date = today + timedelta(days=365 * i / self.freq)
            interest = balance * (self.rate / self.freq)
            principal = pmt - interest
            balance -= principal
            flows.append((pay_date, pmt))

        return flows

    def price(self, yield_rate: float, as_of: date = date.today()) -> float:
        pv = 0.0
        for pay_date, amt in self.cash_flows():
            t = (pay_date - as_of).days / 365
            pv += amt / ((1 + yield_rate) ** t)
        return pv


In [7]:
# ──────────────────────────────────────────────────────────────
# 4) AssetBackedSecurity
# ──────────────────────────────────────────────────────────────
class AssetBackedSecurity(FixedIncomeInstrument):
    """
    A pass-through ABS of multiple amortizing loans.
    """
    def __init__(self, loans: list[AmortizingLoan]):
        self.loans = loans

    def cash_flows(self) -> list[tuple[date, float]]:
        flow_dict: dict[date, float] = {}
        for loan in self.loans:
            for dt, amt in loan.cash_flows():
                flow_dict[dt] = flow_dict.get(dt, 0.0) + amt
        return sorted(flow_dict.items())

    def price(self, yield_rate: float, as_of: date = date.today()) -> float:
        pv = 0.0
        for pay_date, amt in self.cash_flows():
            t = (pay_date - as_of).days / 365
            pv += amt / ((1 + yield_rate) ** t)
        return pv


## module test

In [8]:
# ──────────────────────────────────────────────────────────────
# 5) Script example
# ──────────────────────────────────────────────────────────────
if __name__ == "__main__":
    # 1) Bullet Bond example
    today     = date.today()
    maturity  = today.replace(year=today.year + 5)
    b = BulletBond(par=1000, coupon_rate=0.06, maturity=maturity, freq=2)

    print("----- Bullet Bond -----")
    print(f"Price (@ YTM=5%):     {b.price(0.05, today):.2f}")
    print(f"YTM (@ Price=950.00): {b.yield_to_maturity(950.0):.4f}\n")

    # 2) Amortizing loan example
    loan = AmortizingLoan(principal=200_000, rate=0.04, term_years=30, freq=12)
    print("----- Amortizing Loan -----")
    print(f"Price (@ yield=4%):    {loan.price(0.04, today):.2f}")
    print(f"YTM (@ price=200k):     {loan.yield_to_maturity(200_000):.4f}\n")

    # 3) ABS example (pool of 3 identical loans)
    abs_pool = AssetBackedSecurity([loan, loan, loan])
    print("----- Asset‐Backed Security -----")
    print(f"Price (@ yield=4%):    {abs_pool.price(0.04, today):.2f}")
    print(f"YTM (@ price=3×200k):   {abs_pool.yield_to_maturity(3*200_000):.4f}")


----- Bullet Bond -----
Price (@ YTM=5%):     1076.26
YTM (@ Price=950.00): 0.0812

----- Amortizing Loan -----
Price (@ yield=4%):    201748.19
YTM (@ price=200k):     0.0407

----- Asset‐Backed Security -----
Price (@ yield=4%):    605244.57
YTM (@ price=3×200k):   0.0407


## bond duration & convexity

In [13]:
import numpy as np

def calculate_bond_durations(cash_flows, yield_rate):
    # Calculate Present Value of each cash flow
    present_values = [cf / (1 + yield_rate) ** t for t, cf in enumerate(cash_flows)] #list comp to store
    

    # Total Present Value
    total_pv = sum(present_values)

    # Calculate Macaulay Duration = generator - only needed to cal sum() but not store
    macaulay_duration = sum(t * pv for t, pv in enumerate(present_values)) / total_pv 

    # Calculate Modified Duration
    modified_duration = macaulay_duration / (1 + yield_rate)

    # Calculate Percentage Change for a 1% change in yield
    percentage_change = -modified_duration * 0.01
    
    # Calculate Convexity
    convexity = sum(t * (t + 1) * pv for t, pv in enumerate(present_values)) / total_pv

    return {
        "Macaulay Duration": round(macaulay_duration, 2),
        "Modified Duration": round(modified_duration, 2),
        "Percentage Change": round(percentage_change, 2),
        "Convexity": round(convexity, 2)
    }



## test sample:

In [14]:
# Sample data
cash_flows = [100, 100, 100, 100, 1100]  # Cash flows for each period
yield_rate = 0.05  # Yield (5%)

durations = calculate_bond_durations(cash_flows, yield_rate)

# Extract convexity from the results
convexity = durations["Convexity"]

# Print the results
print(f"Macaulay Duration: {durations['Macaulay Duration']}")
print(f"Modified Duration: {durations['Modified Duration']}")
print(f"Percentage Change: {durations['Percentage Change']}")
print(f"Convexity: {convexity}")

Macaulay Duration: 3.25
Modified Duration: 3.1
Percentage Change: -0.03
Convexity: 15.56


# Conclusion

This notebook provides a comprehensive overview of fixed-income instruments, their cash flow structures, and essential risk metrics, demonstrating the use of OOP principles for effective financial modeling.
